In [1]:
import sys
import pandas as pd
import numpy as np
import re
import json
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import snowflake.connector
import snowflake
import datetime as dt

In [14]:
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 100
plt.rcParams["figure.figsize"] = (10,8)
plt.style.use('seaborn-darkgrid')

In [16]:
con = snowflake.connector.connect(
    user="tliang@endpointclosing.com",
    account="endpoint",
    authenticator="externalbrowser",
    role="SNOWFLAKE_DATA_ENGINEERING",
    warehouse="DATAENGINEERING_WH"
)
cur = con.cursor()

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...


In [17]:
query = "SELECT * FROM LAKE.SALESFORCE.CONTACT WHERE IS_DELETED = 'FALSE'"
all_contact = cur.execute(query).fetch_pandas_all().reset_index()

In [ ]:
# missing mobile filter

In [23]:
sf_contact = all_contact[(all_contact["MAILING_CITY"] == "San Francisco") | (all_contact["MAILING_CITY"] == "SAN FRANCISCO")]

In [24]:
sf_contact[~sf_contact[["NAME","EMAIL","PHONE","MOBILE_PHONE","ACCOUNT_ID","MAILING_CITY","MAILING_STATE"]].isnull().any(axis=1)]

,index,ID,IS_DELETED,MASTER_RECORD_ID,ACCOUNT_ID,IS_PERSON_ACCOUNT,LAST_NAME,FIRST_NAME,SALUTATION,MIDDLE_NAME,SUFFIX,NAME,RECORD_TYPE_ID,OTHER_STREET,OTHER_CITY,OTHER_STATE,OTHER_POSTAL_CODE,OTHER_COUNTRY,OTHER_LATITUDE,OTHER_LONGITUDE,OTHER_GEOCODE_ACCURACY,MAILING_STREET,MAILING_CITY,MAILING_STATE,MAILING_POSTAL_CODE,MAILING_COUNTRY,MAILING_LATITUDE,MAILING_LONGITUDE,MAILING_GEOCODE_ACCURACY,PHONE,FAX,MOBILE_PHONE,HOME_PHONE,OTHER_PHONE,ASSISTANT_PHONE,REPORTS_TO_ID,EMAIL,TITLE,DEPARTMENT,ASSISTANT_NAME,LEAD_SOURCE,BIRTHDATE,DESCRIPTION,OWNER_ID,HAS_OPTED_OUT_OF_EMAIL,HAS_OPTED_OUT_OF_FAX,DO_NOT_CALL,CREATED_DATE,CREATED_BY_ID,LAST_MODIFIED_DATE,LAST_MODIFIED_BY_ID,SYSTEM_MODSTAMP,LAST_ACTIVITY_DATE,LAST_CUREQUEST_DATE,LAST_CUUPDATE_DATE,LAST_VIEWED_DATE,LAST_REFERENCED_DATE,EMAIL_BOUNCED_REASON,EMAIL_BOUNCED_DATE,IS_EMAIL_BOUNCED,PHOTO_URL,JIGSAW,JIGSAW_CONTACT_ID,INDIVIDUAL_ID,TOTAL_MODUS_TRANSACTION_COUNT_C,CUSTOMER_STATUS_C,MOST_RECENT_CAMPAIGN_C,PRODUCT_DEMO_DATE_C,ROLE_C,KCA_C,LISTING_PRICE_C,KNOCK_WEBHOOK_C,CONVERTED_FROM_LEAD_C,ALTERNATE_EMAIL_C,FA_LOYAL_LEAD_C,PREFERRED_ESCROW_COMPANY_C,PREFERRED_TITLE_COMPANY_C,TOTAL_SALES_C,TOTAL_LISTING_BOOK_SIZE_C,TOTAL_LISTING_COUNT_C,TOTAL_TRANSACTION_COUNT_C,STATUS_C,APP_DOWNLOAD_DATE_C,FSBO_ADDRESS_C,ORDER_ENTRY_NOTES_C,FSBO_LISTING_PRICE_C,EMAIL_BOUNCED_C,FIRST_TASK_DATE_C,FIRST_EVENT_DATE_C,SALES_LOFT_1_MOST_RECENT_CADENCE_NAME_C,DISQUALIFIED_REASONS_C,SALES_LOFT_1_MOST_RECENT_CADENCE_NEXT_STEP_DUE_DATE_C,SALES_LOFT_1_MOST_RECENT_LAST_COMPLETED_STEP_C,EVENT_TYPE_C,PREFERRED_PLATFORM_C,CHECKBOX_C,NMLS_ID_C,REFERRER_C,COMING_SOON_ADDRESS_C,TOTAL_DIRECTED_ENDPOINT_ORDERS_C,PI_NEEDS_SCORE_SYNCED_C,PI_PARDOT_LAST_SCORED_AT_C,PI_CAMPAIGN_C,PI_COMMENTS_C,PI_CONVERSION_DATE_C,PI_CONVERSION_OBJECT_NAME_C,PI_CONVERSION_OBJECT_TYPE_C,PI_CREATED_DATE_C,PI_FIRST_ACTIVITY_C,PI_FIRST_SEARCH_TERM_C,PI_FIRST_SEARCH_TYPE_C,PI_FIRST_TOUCH_URL_C,PI_GRADE_C,PI_LAST_ACTIVITY_C,PI_NOTES_C,PI_PARDOT_HARD_BOUNCED_C,PI_SCORE_C,PI_URL_C,PI_UTM_CAMPAIGN_C,PI_UTM_CONTENT_C,PI_UTM_MEDIUM_C,PI_UTM_SOURCE_C,PI_UTM_TERM_C,AGENT_LICENSE_C,AGENT_MLS_ID_C,AVERAGE_CLOSE_PRICE_C,BS_COUNT_C,BUYER_SELLER_LINK_C,COUNTY_C,TOTAL_NON_DIRECTED_ENDPOINT_ORDERS_C,FA_DIRECTED_ESCROW_TRANSACTIONS_C,FA_DIRECTED_TRANSACTIONS_C,LEAD_RANKING_C,LEVEL_C,LS_SALES_C,LISTING_COUNT_C,MATRIX_UNIQUE_ID_C,PMX_AGENT_ID_C,PREFERRED_ESCROW_CO_C,PREFERRED_ESCROW_SOW_C,PREFERRED_TITLE_SOW_C,SECONDARY_ESCROW_CO_C,SECONDARY_TITLE_CO_C,SIGN_UP_DATE_C,SUPPORT_TEAM_C,BS_SALES_C,TOTAL_DIRECTED_ESCROW_TRANSACTIONS_C,TOTAL_DIRECTED_TITLE_TRANSACTIONS_C,PREFERRED_TITLE_CO_C,ALTERNATE_PHONE_C,_FIVETRAN_SYNCED,DISQUALIFIED_NOTES_C,RETIRING_C,PREFERS_TRADITIONAL_C,NO_CONTACT_C,IN_HOUSE_ESCROW_JV_C,BAD_EXPERIENCE_C,NO_DIRECTING_ORDERS_C,UP_TO_BROKER_C,EMAIL_UNKNOWN_DELIVERABILITY_C,ACCOUNT_MANAGER_C,SALES_DEVELOPMENT_REP_C,RELATIONSHIP_MANAGER_C,WEB_LEAD_C,DO_NOT_CONTACT_C,DEMO_SET_ON_C,INACTIVE_C,CONVERTED_LEAD_ID_C,DEMO_SCHEDULED_FOR_C,FIRST_OPEN_ORDER_DATE_C,LAST_OPEN_ORDER_DATE_C,DEMO_NOT_HELD_C,ONBOARDED_DATE_C,TERRITORY_C,PRODUCT_RE_DEMO_DATE_C,WIN_BACK_CLIENT_C,CANCELLED_FILE_C,INACTIVITY_C,TRANSFERRED_BDM_C,JET_CLOSING_C,NO_SHOW_CP_C,MEETING_TYPE_CP_C,BOOKING_STATUS_CP_C,TOTAL_ACTIVE_LISTINGS_C,ACTIVE_LISTINGS_SINCE_LAST_OPEN_ORDER_C,TOTAL_LISTINGS_C
153,148,0038V00002Wdo9fQAB,False,None,0011H00002316f6QAA,False,Marguleas,Anthony,None,None,None,Anthony Marguleas,0121H000000zyGqQAI,None,None,None,None,None,NaN,NaN,None,466 Brannan Street,San Francisco,CA,94107,None,NaN,NaN,None,415-525-4913,None,(310) 804-1362,None,None,None,None,anthony@amalfiestates.com,None,None,None,Transferred Client,None,None,0058V00000C7zvSQAR,True,False,False,2022-06-13 11:06:20-07:00,0058V00000C7zvSQAR,2022-06-14 17:24:27-07:00,0051H00000BQg3LQAT,2022-06-14 17:24:27-07:00,2022-06-13,NaT,NaT,NaT,NaT,None,NaT,False,/services/images/photo/0038V00002Wdo9fQAB,None,None,None,NaN,Committed,7018V000000HFNsQAO,None,Agent,False,NaN,False,True,None,None,None,None,NaN,NaN,NaN,NaN

In [19]:
# all_contact["MAILING_CITY"].unique()

array(['MESA', 'GOODYEAR', 'Seattle', 'Kirkland', 'San Diego', 'Ventura',
       'San Antonio', 'San Francisco', 'Scottsdale', 'Gilbert',
       'Royal Palm Beach', 'Gig Harbor', 'Kent', 'Torrance', 'Phoenix',
       'GLENDALE', 'Mesa', 'PHOENIX', 'Bellevue', 'St. Petersburg',
       'SCOTTSDALE', 'Dallas', 'Temecula', 'Austin', 'Newport Beach',
       'Tempe', 'Playa Del Rey', 'Lake Tapps', 'Marina Del Rey',
       'Salinas', 'Pasadena', 'Rolling Hills Estates', 'Beverly Hills',
       'Webster', 'Irvine', 'Felton', 'Benbrook', 'San Jose',
       'Federal Way', 'Coppell', 'Tukwila', 'CHANDLER', 'Camarillo',
       'Montecito', 'Puyallup', 'PRESCOTT', 'Tacoma', 'Fort Worth',
       'Bothell', 'Frisco', 'Houston', 'PICO RIVERA', 'Dana Point',
       'Visalia', 'League City', 'Los Angeles', 'Mill Creek', 'San Mateo',
       'Arcadia', 'Calabasas', 'Southlake', None, 'Santa Rosa',
       'Sun Lakes', 'Marysville', 'Lakewood', 'PEORIA', 'Glendale',
       'Manhattan Beach', 'Westlake Villa

In [2]:
sample_contact = pd.read_csv("../data/sample_contact.csv")

In [5]:
sample_contact

,Unnamed: 0,index,ID,IS_DELETED,MASTER_RECORD_ID,ACCOUNT_ID,IS_PERSON_ACCOUNT,LAST_NAME,FIRST_NAME,SALUTATION,...,ONBOARDED_DATE_C,TERRITORY_C,PRODUCT_RE_DEMO_DATE_C,WIN_BACK_CLIENT_C,CANCELLED_FILE_C,INACTIVITY_C,TRANSFERRED_BDM_C,RECENT_CONTACT,RECENT_CONTACT_QUARTER,MAILING_STATE_STD
0,449,292,0038V00002VO2lDQAT,False,NaN,0018V00002M8DaCQAV,False,Graves,Monica,NaN,...,NaN,NaN,NaN,False,False,False,False,False,True,CO
1,487,330,0038V00002Vbk8cQAB,False,NaN,0011H00002HEbOZQA1,False,Fox,Tristan,NaN,...,NaN,NaN,NaN,False,False,False,False,True,True,NaN
2,1453,638,0038V00002VNyZJQA1,False,NaN,0018V00002M841DQAR,False,Overton,Barry,NaN,...,NaN,NaN,NaN,False,False,False,False,False,True,CO
3,1510,695,0038V00002VNKDLQA5,False,NaN,0018V00002Ku3NgQAJ,False,Germano,Tom,NaN,...,2022-03-23,NaN,NaN,False,False,False,False,False,True,FL
4,4197,1914,0038V00002VMQOvQAP,False,NaN,0018V00002KsWlOQAV,False,Collier,Michael,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,True,FL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,6969,1824,0031H00002SANzXQAX,False,NaN,0011H000025Kr7nQAC,False,Garcia,Reyna,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,True,CA
1996,7047,1902,0031H00002MdEOaQAN,False,NaN,0011H0000230xXkQAI,False,Brennan,Coleen,Mrs.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,True,CA
1997,9234,4089,0038V00002UPQAIQA5,False,NaN,0011H0000230y3YQAQ,False,Karta,Rose,NaN,...,2021-11-16,NaN,NaN,False,False,False,False,False,True,CA
1998,10479,1098,0038V00002URmmSQAT,False,NaN,0011H00002GEVD3QAP,False,Branstrom,Lexi,NaN,...,NaN,NaN,NaN,False,False,False,False,True,True,WA


In [13]:
sample_contact[sample_contact["MAILING_CITY"] == "San Francisco"]

,Unnamed: 0,index,ID,IS_DELETED,MASTER_RECORD_ID,ACCOUNT_ID,IS_PERSON_ACCOUNT,LAST_NAME,FIRST_NAME,SALUTATION,...,ONBOARDED_DATE_C,TERRITORY_C,PRODUCT_RE_DEMO_DATE_C,WIN_BACK_CLIENT_C,CANCELLED_FILE_C,INACTIVITY_C,TRANSFERRED_BDM_C,RECENT_CONTACT,RECENT_CONTACT_QUARTER,MAILING_STATE_STD
12,9338,4193,0038V00002TRQLEQA5,False,NaN,0011H00002H4kvSQAR,False,Burns,Josh,NaN,...,NaN,NaN,NaN,False,NaN,NaN,NaN,True,True,CA
18,6587,1442,0038V00002USDVDQA5,False,NaN,0011H00002HilAbQAJ,False,Silva,Camila,NaN,...,NaN,NaN,NaN,False,False,False,False,False,True,CA
26,1461,646,0038V00002W5VLqQAN,False,NaN,0011H00002FpjmFQAR,False,Nassano,Carey,NaN,...,2022-05-18,NaN,NaN,False,False,False,False,True,True,CA
78,10274,893,0038V00002USDWGQA5,False,NaN,0011H00002HilAbQAJ,False,Khalfi,Lori,NaN,...,NaN,NaN,NaN,False,False,False,False,False,True,CA
87,9727,346,0038V00002UPBC7QAP,False,NaN,0011H00002HPow2QAD,False,Leon,Martin,NaN,...,NaN,NaN,NaN,False,False,False,False,True,True,CA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1826,6588,1443,0038V00002USDciQAH,False,NaN,0011H00002HilAbQAJ,False,Vettese,Frank,NaN,...,NaN,NaN,NaN,False,False,False,False,False,True,CA
1866,6088,943,0038V00002W2y15QAB,False,NaN,0011H00002HETo6QAH,False,Beattie,Sean,NaN,...,NaN,NaN,NaN,False,False,False,False,True,True,CA
1941,6579,1434,0038V00002VNZhMQAX,False,NaN,0011H00002HPovtQAD,False,Branchini,Jennifer,NaN,...,NaN,NaN,NaN,False,False,False,False,False,True,CA
1963,1416,601,0038V00002WEudrQAD,False,NaN,0011H00002FpjmFQAR,False,Swann,Rachel,NaN,...,2022-05-19,NaN,NaN,False,False,False,False,True,True,CA
